In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds 
import numpy as np
import matplotlib.pyplot as plt 
#import cv2

from tensorflow import keras

In [4]:
# Process the data
train_ds, test_ds = tfds.load('visual_domain_decathlon/gtsrb', split = ['train', 'test'], data_dir='D:\\tensorflow_datasets', shuffle_files=True, as_supervised = True)

C:\Python\anaconda3\envs\tf-env\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Python\anaconda3\envs\tf-env\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


NonMatchingChecksumError: Artifact http://www.robots.ox.ac.uk/~vgg/share/decathlon-1.0-devkit.tar.gz, downloaded to D:\tensorflow_datasets\downloads\robo.ox.ac.uk_vgg_shar_deca-1.0-devkkd2n60sUY-vJDHd0ga10OR7nxGl1HK3LWcRePkBpp84.tar.gz.tmp.7f1d3747c0c046a18fa01258b8dcd989\decathlon-1.0-devkit.tar.gz, has wrong checksum.

In [ ]:
train_ds = tfds.as_numpy(train_ds)
test_ds = tfds.as_numpy(test_ds)

In [ ]:
def create_img_label_set(data_set):
    image_set = []
    label_set = []
    for img in data_set:
       image, label = img[0], img[1]
       image_set.append(image)
       label_set.append(label)
    return np.asarray(image_set), np.asarray(label_set)

In [ ]:
img_train, label_train = create_img_label_set(train_ds)
img_test, label_test = create_img_label_set(test_ds)

In [ ]:
print('Train images size:', img_train.size)
print('Train labels shape:', label_train.shape)
print('Test images shape:', img_test.shape)
print('Test labels shape:', label_test.shape)